In [1]:
from differential_evolution import optimize
from timer import Timer
import torch
import torch.nn.functional as F
from time import sleep
from itertools import permutations
from fastprogress.fastprogress import master_bar

In [2]:
torch.cuda.set_device(1)

In [3]:
def pairwise_distances(A,B):
    # Both A and B should be (bs,n,d) where n is some number of points and d is dimension (e.g. 2 for R^2)
    w = (A[:,:,None] - B[:,None])
    d2 = (w*w).sum(dim = -1)
    return torch.sqrt(d2)

def inverse_perm(P):
    Q = [0]*len(P)
    for i,p in enumerate(P):
        Q[p] = i
    return Q

def too_close_loss(D):
    u = D.shape[1]
    i = torch.arange(u,device=D.device)
    D[:,i,i] = 1
    β = (D < 0.0001).double()
    return (β).sum(dim=(1,2))

def distances_too_similar(D):
    U = pairwise_distances(D[...,None],D[...,None])
    u = U.shape[1]

    i = torch.arange(u,device=U.device)
    U[:,i,i] = 1
    U = U.view(U.shape[0],-1)
    β = (U < 0.00001).double()
    return (β).sum(dim=1)

        
def leo_cost(A):
    # A.shape = (bs, numpoints, d)
    bs,n,d = A.shape
    
    D = pairwise_distances(A,A)
    
    max_chicos = D[:,:d,d:].reshape(bs,-1).max(dim=1)[0]
    
    max_medianos = D[:,d:,d:].reshape(bs,-1).max(dim=1)[0]
    I = torch.arange(n, device=A.device)
    D[:,I,I] = 10000
    
    min_medianos = D[:,d:,d:].reshape(bs,-1).min(dim=1)[0]

    min_grandes = D[:,:d,:d].reshape(bs,-1).min(dim=1)[0]
    
    #print(f"min_grandes={min_grandes}")
    #print(f"max_medianos={max_medianos}")
    #print(f"min_medianos={min_medianos}")
    #print(f"max_chicos={max_chicos}")
    #print(f"{max_chicos.shape}")
    #print(f"{max_medianos.shape}")
    #print(f"{min_medianos.shape}")
    #print(f"{min_grandes.shape}")
    #print(f"{F.relu(max_chicos - min_medianos)}")
    #print(f"{F.relu(max_medianos-min_grandes)}")
    #print(f"{too_close_loss(D)}")
    #print(f"{distances_too_similar(D)}")
    
    return 100*(F.relu(max_chicos - min_medianos) + F.relu(max_medianos-min_grandes)) + too_close_loss(D) + distances_too_similar(D)
    

In [4]:
def proyectar(x):
    bs,n,d = x.shape
    x[:,:d,-1] = 0
    return torch.clamp(x,-100,100)

A=[[2, 0, 0, 0, 0, 0], 
   [0, 2, 0, 0, 0, 0], 
   [0, 0, 2, 0, 0, 0], 
   [0, 0, 0, 2, 0, 0], 
   [0, 0, 0, 0, 2, 0], 
   [0, 0, 0, 0, 0, 2],
   [1.3, 1.3, -0.1, -0.1, -0.1, -0.1], 
   [-0.1, -0.1, 1.3, 1.3, -0.1, -0.1], 
   [-0.1, -0.1, -0.1, -0.1, 1.3, 1.3]]

A=torch.tensor(A)
#A += 0.01*torch.randn_like(A)

A =torch.tensor([[-1.5094,  0.8203, -2.2727,  2.0785],
        [ 1.7838,  0.8836,  1.7320,  1.9060],
        [-0.4184, -1.6662, -3.0278, -2.1233],
        [ 3.7668, -1.3022, -0.3410, -2.0159],
        [-0.8384, -1.7425,  0.6483, -0.7001],
        [ 2.4868, -1.2303, -2.2418,  1.4634],
        [ 0.9739,  2.3999, -1.2020, -1.3232]])

A.shape

pairwise_distances(A[None],A[None])

leo_cost(A[None])

In [ ]:
d = 3

shuffles = 2
epochs = 200000
num_populations = 64
pop_size = 32
bs = num_populations*pop_size
device = 'cuda'

simulaciones = 5

mb = master_bar(range(simulaciones))
malas = []
for _ in mb:
    initial_pop = torch.randn((bs,d+3,d),device=device).double()
    
    #for i in range(bs):
        #initial_pop[i,:,:] = A + 0.001*torch.randn_like(A)
    
    value, x = optimize(leo_cost, 
                        num_populations=num_populations, 
                        initial_pop=initial_pop, 
                        epochs=epochs, 
                        shuffles=shuffles,
                        proj_to_domain=proyectar, #+ torch.randn_like(x)*0.00001, 
                        use_cuda=False,
                        break_at_cost=0,
                        mb=mb
                       )
    try:
        sleep(0.4)
    except KeyboardInterrupt:
        raise
    
    if value == 0:
        print(f"BAD: \n{x}, best value: {value}\n")
        D = pairwise_distances(x[None],x[None])
        max_chicos = D[:,:d,d:].reshape(1,-1).max(dim=1)[0]
    
        max_medianos = D[:,d:,d:].reshape(1,-1).max(dim=1)[0]
        I = torch.arange(d+3, device=x.device)
        D[:,I,I] = 10000
        
        min_medianos = D[:,d:,d:].reshape(1,-1).min(dim=1)[0]
    
        min_grandes = D[:,:d,:d].reshape(1,-1).min(dim=1)[0]
        
        print(f"Distances matrix: \n{D}\n")
        
        print(f"max_chicos={max_chicos}")
        print(f"min_medianos={min_medianos}")
        print(f"max_medianos={max_medianos}")
        print(f"min_grandes={min_grandes}\n\n")
        
        malas.append(x)
    else:
        print(f"Esto es lo mejor que logré: {x}, con valor {value}")

In [ ]:
value

In [ ]:
print(f"% de malas: {len(malas)/simulaciones}")

In [ ]:
A=torch.rand(4,2,2);A